## Dataloader

In [1]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def create_train_val_test_dataloaders(data_dir, batch_size, num_workers=4):
    # Define specific transformations for each dataset
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize((244, 244)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]),
        'val': transforms.Compose([
            transforms.Resize((244, 244)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]),
    }

    # Create datasets using specific transformations
    image_datasets = {
        'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train']),
        'val': datasets.ImageFolder(os.path.join(data_dir, 'val'), data_transforms['val'])
    }

    # Create dataloaders
    dataloaders = {
        'train': DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True, num_workers=num_workers),
        'val': DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=False, num_workers=num_workers)
    }
    return dataloaders

data_dir = 'Dataset/'
batch_size = 32
dataloaders = create_train_val_test_dataloaders(data_dir, batch_size)

In [2]:
import torch
# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Load Models

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

def load_resnet50(num_classes):
    model =  models.resnet50(weights='ResNet50_Weights.DEFAULT')
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)
    return model

def load_efficientnet_b0(num_classes):
    model = models.efficientnet_b0(weights='EfficientNet_B0_Weights.DEFAULT')
    num_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_features, num_classes)
    return model

def load_mobilenet_v3_large(num_classes):
    model = models.mobilenet_v3_large(weights='MobileNet_V3_Large_Weights.DEFAULT')
    num_features = model.classifier[3].in_features
    model.classifier[3] = nn.Linear(num_features, num_classes)
    return model

## Training

In [5]:
from tqdm import tqdm
import torch
import os

def train_model(model, dataloaders, criterion, optimizer, num_epochs, save_dir):
    train_loader = dataloaders['train']
    test_loader = dataloaders['val']

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_train_loss = 0.0
        correct_train = 0
        total_train = 0

        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} - Training'):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels.float())

            running_train_loss += loss.item() * inputs.size(0)

            # Calculate accuracy
            predicted = torch.round(torch.sigmoid(outputs))
            correct_train += (predicted == labels.float()).sum().item()
            total_train += labels.size(0)

        train_loss = running_train_loss / len(train_loader.dataset)
        train_acc = correct_train / total_train

        # Validation phase
        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for inputs, labels in tqdm(test_loader, desc=f'Epoch {epoch+1}/{num_epochs} - Validation'):
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs).squeeze()
                loss = criterion(outputs, labels.float())

                running_val_loss += loss.item() * inputs.size(0)

                # Calculate accuracy
                predicted = torch.round(torch.sigmoid(outputs))
                correct_val += (predicted == labels.float()).sum().item()
                total_val += labels.size(0)

        val_loss = running_val_loss / len(test_loader.dataset)
        val_acc = correct_val / total_val

        # Print epoch statistics
        print(f'Epoch [{epoch+1}/{num_epochs}] - '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%')

        # Save the model after each epoch
        torch.save(model.state_dict(), os.path.join(save_dir, f'model_epoch_{epoch+1}.pth'))

    print('Training complete.')

## EfficientNet

In [6]:
model = load_efficientnet_b0(num_classes=1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, dataloaders, criterion, optimizer, num_epochs=10, save_dir='Checkpoints/efficientnet')

Epoch 1/10 - Validation: 100%|█████████████████████████████████████████████████████████| 94/94 [00:18<00:00,  5.18it/s]


Epoch [1/10] - Train Loss: 0.2145, Train Acc: 91.24%, Val Loss: 0.0762, Val Acc: 97.50%


Epoch 2/10 - Validation: 100%|█████████████████████████████████████████████████████████| 94/94 [00:14<00:00,  6.53it/s]


Epoch [2/10] - Train Loss: 0.0736, Train Acc: 97.23%, Val Loss: 0.1240, Val Acc: 96.07%


Epoch 3/10 - Validation: 100%|█████████████████████████████████████████████████████████| 94/94 [00:13<00:00,  6.74it/s]


...

Epoch [8/10] - Train Loss: 0.0212, Train Acc: 99.20%, Val Loss: 0.0517, Val Acc: 98.40%


Epoch 9/10 - Validation: 100%|█████████████████████████████████████████████████████████| 94/94 [00:13<00:00,  6.73it/s]


Epoch [9/10] - Train Loss: 0.0229, Train Acc: 99.17%, Val Loss: 0.0422, Val Acc: 98.27%


Epoch 10/10 - Validation: 100%|████████████████████████████████████████████████████████| 94/94 [00:13<00:00,  6.88it/s]

Epoch [10/10] - Train Loss: 0.0207, Train Acc: 99.29%, Val Loss: 0.0472, Val Acc: 98.23%
Training complete.


## Mobilenet_v3_large

In [7]:
model = load_mobilenet_v3_large(num_classes=1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, dataloaders, criterion, optimizer, num_epochs=10, save_dir='Checkpoints/mobilenet_v3_large')

Epoch 1/10 - Validation: 100%|█████████████████████████████████████████████████████████| 94/94 [00:12<00:00,  7.33it/s]


Epoch [1/10] - Train Loss: 0.2054, Train Acc: 91.96%, Val Loss: 0.6805, Val Acc: 85.60%


Epoch 2/10 - Validation: 100%|█████████████████████████████████████████████████████████| 94/94 [00:12<00:00,  7.33it/s]


Epoch [2/10] - Train Loss: 0.0647, Train Acc: 97.57%, Val Loss: 0.7714, Val Acc: 82.57%


Epoch 3/10 - Validation: 100%|█████████████████████████████████████████████████████████| 94/94 [00:12<00:00,  7.53it/s]


...

Epoch [8/10] - Train Loss: 0.0148, Train Acc: 99.49%, Val Loss: 0.3298, Val Acc: 91.13%


Epoch 9/10 - Validation: 100%|█████████████████████████████████████████████████████████| 94/94 [00:12<00:00,  7.50it/s]


Epoch [9/10] - Train Loss: 0.0251, Train Acc: 99.06%, Val Loss: 0.0642, Val Acc: 97.47%


Epoch 10/10 - Validation: 100%|████████████████████████████████████████████████████████| 94/94 [00:12<00:00,  7.56it/s]

Epoch [10/10] - Train Loss: 0.0189, Train Acc: 99.26%, Val Loss: 0.1348, Val Acc: 96.07%
Training complete.


## ResNet50

In [8]:
model = load_resnet50(num_classes=1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, dataloaders, criterion, optimizer, num_epochs=10, save_dir='Checkpoints/resnet50')

Epoch 1/10 - Validation: 100%|█████████████████████████████████████████████████████████| 94/94 [00:16<00:00,  5.62it/s]


Epoch [1/10] - Train Loss: 0.3564, Train Acc: 84.16%, Val Loss: 0.2744, Val Acc: 89.00%


Epoch 2/10 - Validation: 100%|█████████████████████████████████████████████████████████| 94/94 [00:16<00:00,  5.55it/s]


Epoch [2/10] - Train Loss: 0.1667, Train Acc: 93.37%, Val Loss: 1.2115, Val Acc: 58.17%


Epoch 3/10 - Validation: 100%|█████████████████████████████████████████████████████████| 94/94 [00:16<00:00,  5.55it/s]


...

Epoch [8/10] - Train Loss: 0.0362, Train Acc: 98.44%, Val Loss: 0.1561, Val Acc: 94.90%


Epoch 9/10 - Validation: 100%|█████████████████████████████████████████████████████████| 94/94 [00:16<00:00,  5.55it/s]


Epoch [9/10] - Train Loss: 0.0445, Train Acc: 98.30%, Val Loss: 0.2080, Val Acc: 92.73%


Epoch 10/10 - Validation: 100%|████████████████████████████████████████████████████████| 94/94 [00:17<00:00,  5.51it/s]

Epoch [10/10] - Train Loss: 0.0290, Train Acc: 98.99%, Val Loss: 0.3451, Val Acc: 90.70%
Training complete.
